In [2]:
import random
import datetime
from subprocess import check_output, PIPE, Popen
import subprocess
import shlex
import threading
import os

class Gateway():
    address = ""
    latency = 0.0
    ts = None
    status = True
    def __init__(self, address, latency, ts, status = True):
        self.address = address
        self.latency = latency
        self.ts = ts
        self.status = status
        
    def setStatus(self, status):
        self.status=status
    
    def getStatus(self):
        return self.status
    
    def changeInformation(self, latency, ts):
        self.latency = latency
        self.ts = ts
    
    def getLatency(self):
        return self.latency
    
    def getTimestamp(self):
        return self.ts

# Client node information    
class Client():
    address = ""
    neighbours = []
    gateways = []
    defaultGateway = None
    round = 0
    
    def __init__(self, address, neighbours = [], gateways = [], defaultGateway = None):
        self.address = address
        self.neighbours = neighbours
        self.gateways = gateways
        self.defaultGateway = defaultGateway
    
    def getDefaultGateway(self):
        return self.defaultGateway
    
    def setDefaultGateway(self, gw):
        self.defaultGateway = gw
    
    def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)
    
    def removeNeighbour(self, neighbour):
        self.neighbours.remove(neighbour)
    
    def addGateway(self, gateway):
        self.gateways.append(gateway)
    
    def removeGateway(self, gateway):
        if(gateway in self.gateways):
            self.gateways.remove(gateway)
    
    def updateGateway(self, gateway):
        gw = next(x for x in self.gateways if x.address == gateway.address)
    
    def printGatewayInformation(self):
        print('Total number:', len(self.gateways))
        for gw in self.gateways:
            print(gw.address,':',gw.latency,':', gw.ts,':', gw.status)
    
    def pingGateway(self, gateway):        
        cmd='''curl -x '''+gateway.address+''':3128 -U david.pinilla:"|Jn 5DJ\\7inbNniK|m@^ja&>C" -m 180 -w %{time_total},%{http_code} http://ovh.net/files/1Mb.dat -o /dev/null -s'''
        command = Popen(shlex.split(cmd),stdout=PIPE, stderr=PIPE)
        stdout, stderr = command.communicate()
        lat, status = stdout.decode("utf-8").split(',')
        if(int(status) == 0):            
            with open('log_output','a') as f:                
                f.write(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")+', removing:'+gateway.address+'\n')
            self.removeGateway(gateway)
        elif(int(status)!=200):
            gateway.ts = datetime.datetime.now()
            gateway.latency = 200
            self.setCategory(gateway)
        else:
            gateway.ts = datetime.datetime.now()
            gateway.latency = float(lat)      
            self.setCategory(gateway)
    
    def pingTest(self, address):
        response = subprocess.call(['ping', '-q', '-c', '1', '-w','5',address], stdout=subprocess.DEVNULL)
        if response == 0:
            return True
        else:
            return False
    
    
    def setCategory(self, gw):        
        if gw.latency < 3:
            gw.status= True
        else:
            gw.status = False
        
        
    def selectBest(self):
        selected = self.defaultGateway
        for gw in self.gateways:
            if gw.latency < selected.latency:
                selected = gw                
        self.defaultGateway = selected
    
    def senseGateways(self):        
        threading.Timer(300.0, self.senseGateways).start()
        for gw in self.gateways:
            self.pingGateway(gw)
        self.selectBest()
        self.printGatewayInformation()
        
    def connect(self):
        threading.Timer(60.0, self.connect).start()
        print('Connecting to: '+self.defaultGateway.address)
        cmd='''curl -x '''+self.defaultGateway.address+''':3128 -U david.pinilla:"|Jn 5DJ\\7inbNniK|m@^ja&>C" -m 180 -w %{time_starttransfer},%{time_total},%{http_code} http://ovh.net/files/1Mb.dat -o /dev/null -s'''
        command = Popen(shlex.split(cmd),stdout=PIPE, stderr=PIPE)
        stdout, stderr = command.communicate()
        ttfb, lat, status = stdout.decode("utf-8").split(',')
        with open('download_best','a') as f:
            f.write("{0},{1},{2},{3},{4}\n".format(datetime.datetime.now(),self.round,
                                                   self.defaultGateway.address,float(lat),int(status)))
        self.round += 1
                      
node1 = Gateway('10.139.40.85', 0.15, '2018 Feb 15 18:59:15', False)
node2 = Gateway('10.139.40.122', 0.20, '2018 Feb 15 18:20:15',False)
node3 = Gateway('10.138.57.2', 0.02, '2018 Feb 15 18:45:07', False)
node4 = Gateway('10.228.0.83', 0.5, '2018 Feb 15 18:47:21', False)
node5 = Gateway('10.138.77.2', 0.78, '2018 Feb 15 18:26:19', False)
node6 = Gateway('10.138.85.130', 0.35, '2018 Feb 15 19:00:01', True)
node7 = Gateway('10.139.17.4', 2.1, '2018 Feb 15 18:26:04', False)
node8 = Gateway('10.139.37.194', 1.5, '2018 Feb 15 18:30:14', True)
node9 = Gateway('10.138.62.2', 0.04, '2018 Feb 15 19:47:59', False)
node10 = Gateway('10.138.25.67', 0.09, '2018 Feb 15 17:59:07', False)


client4= Client('10.139.40.77', [], [node1,node2,node4, node8, node9, node10, node6,node7, node5, node3], node7)
for gw in client4.gateways:
    if client4.pingTest(gw.address) is False:
        client4.removeGateway(gw)
client4.senseGateways()
client4.connect()

Total number: 7
10.139.40.85 : 0.029016 : 2018-03-13 12:09:32.730714 : True
10.139.40.122 : 0.026495 : 2018-03-13 12:09:32.765325 : True
10.139.37.194 : 0.118299 : 2018-03-13 12:09:32.892708 : True
10.138.25.67 : 0.046162 : 2018-03-13 12:09:32.947337 : True
10.138.85.130 : 0.226905 : 2018-03-13 12:09:33.181625 : True
10.139.17.4 : 200 : 2018-03-13 12:09:33.269358 : False
10.138.57.2 : 0.038739 : 2018-03-13 12:09:33.317441 : True
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-1-f2ba850b8f09>", line 120, in senseGateways
    self.pingGateway(gw)
  File "<ipython-input-1-f2ba850b8f09>", line 89, in pingGateway
    selg.setCategory(gateway)
NameError: name 'selg' is not defined



Connecting to: 10.139.40.122
Total number: 7
10.139.40.85 : 0.028616 : 2018-03-13 12:14:32.730543 : True
10.139.40.122 : 0.028248 : 2018-03-13 12:14:32.767240 : True
10.139.37.194 : 0.119031 : 2018-03-13 12:14:32.894885 : True
10.138.25.67 : 0.031974 : 2018-03-13 12:14:32.935458 : True
10.138.85.130 : 0.200218 : 2018-03-13 12:14:33.143319 : True
10.139.17.4 : 200 : 2018-03-13 12:14:33.209487 : False
10.138.57.2 : 0.032192 : 2018-03-13 12:14:33.250039 : True
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-1-f2ba850b8f09>", line 120, in senseGateways
    self.pingGateway(gw)
  File "<ipython-input-1-f2ba850b8f09>", line 89, in pingGateway
    selg.setCategory(gateway)
NameError: name 'selg' is not defined



Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Total number: 7
10.139.40.85 : 0.031867 : 2018-03-13 12:19:32.734395 : True
10.139.40.122 : 0.02725 : 2018-03-13 12:19:32.770107 : True
10.139.37.194 : 0.13097 : 2018-03-13 12:19:32.909498 : True
10.138.25.67 : 0.030915 : 2018-03-13 12:19:32.949045 : True
10.138.85.130 : 0.212508 : 2018-03-13 12:19:33.170671 : True
10.139.17.4 : 200 : 2018-03-13 12:19:33.281487 : False
10.138.57.2 : 0.050766 : 2018-03-13 12:19:33.340639 : True
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122


Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-1-f2ba850b8f09>", line 120, in senseGateways
    self.pingGateway(gw)
  File "<ipython-input-1-f2ba850b8f09>", line 89, in pingGateway
    selg.setCategory(gateway)
NameError: name 'selg' is not defined



Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Total number: 7
10.139.40.85 : 0.03214 : 2018-03-13 12:24:32.735090 : True
10.139.40.122 : 0.030943 : 2018-03-13 12:24:32.774687 : True
10.139.37.194 : 0.121596 : 2018-03-13 12:24:32.903024 : True
10.138.25.67 : 0.654697 : 2018-03-13 12:24:33.566493 : True
10.138.85.130 : 0.218541 : 2018-03-13 12:24:33.790896 : True
10.139.17.4 : 200 : 2018-03-13 12:24:33.897756 : False
10.138.57.2 : 0.047354 : 2018-03-13 12:24:33.953803 : True
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Connecting to: 10.139.40.122


Exception in thread Thread-24:
Traceback (most recent call last):
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/khulan/anaconda3/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "<ipython-input-1-f2ba850b8f09>", line 120, in senseGateways
    self.pingGateway(gw)
  File "<ipython-input-1-f2ba850b8f09>", line 89, in pingGateway
    selg.setCategory(gateway)
NameError: name 'selg' is not defined



Connecting to: 10.139.40.122
Connecting to: 10.139.40.122
Total number: 7
10.139.40.85 : 0.032591 : 2018-03-13 12:29:32.735913 : True
10.139.40.122 : 0.033619 : 2018-03-13 12:29:32.777907 : True
10.139.37.194 : 0.121501 : 2018-03-13 12:29:32.907757 : True
10.138.25.67 : 0.04225 : 2018-03-13 12:29:32.958790 : True
10.138.85.130 : 0.222801 : 2018-03-13 12:29:33.190448 : True
10.139.17.4 : 200 : 2018-03-13 12:29:33.272691 : False
10.138.57.2 : 0.111871 : 2018-03-13 12:29:33.393654 : True
Connecting to: 10.139.40.85
Connecting to: 10.139.40.85
Connecting to: 10.139.40.85
Total number: 7
10.139.40.85 : 0.031597 : 2018-03-13 12:33:12.448257 : True
10.139.40.122 : 6.306855 : 2018-03-13 12:33:18.763780 : False
10.139.37.194 : 6.846912 : 2018-03-13 12:33:25.619973 : False
10.138.25.67 : 0.036148 : 2018-03-13 12:33:25.664961 : True
10.138.85.130 : 0.226173 : 2018-03-13 12:33:25.899552 : True
10.139.17.4 : 3.416837 : 2018-03-13 12:33:29.325252 : False
10.138.57.2 : 0.043352 : 2018-03-13 12:33:29.